In [1]:
!pip install transformers -q
!pip install wandb -q
#!pip install datasets
!pip install nlp
#!pip install rouge_score
!pip install rouge
#!curl -q https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

     |████████████████████████████████| 1.7 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 13.4 MB 7.2 MB/s eta 0:00:01
     |████████████████████████████████| 81 kB 11.5 MB/s eta 0:00:01


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import time

# Importing the T5 modules from huggingface/transformers
# T5ForConditionalGeneration is specific for sequence-to-sequence
from transformers import T5Tokenizer, T5ForConditionalGeneration

#from nlp import load_metric
import nlp
from rouge import Rouge

import wandb

ModuleNotFoundError: No module named 'torch'

In [ ]:
# # Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
# !wandb login


In [ ]:
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True

train_size = 0.8

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#/content/drive/My Drive/W266/data/ICSI_extrac_abstrac_512token.csv

In [ ]:
# id = wandb.util.generate_id()
# id
#dwlkfpg3 AMI 1024
#1aei9r6r ICSI 1024
#3ugok7an ICSI 512
#30e6cuxp AMI 512

In [ ]:
# #run = wandb.init(project="T5_1024_MSFT_AMI_01",resume=True)
# run = wandb.init(project="T5_1024_MSFT_AMI_01", id="dwlkfpg3", resume="allow")

# config = wandb.config          # Initialize config
# config.TRAIN_BATCH_SIZE = 1    # input batch size for training (default: 64)
# config.VALID_BATCH_SIZE = 1    # input batch size for testing (default: 1000)
# config.EPOCHS = 50        # number of epochs to train (default: 10)
# config.LEARNING_RATE = 0.0005   # learning rate (default: 0.01)
# config.SEED = 42               # random seed (default: 42)
# config.MAX_LEN = 1024
# config.SUMMARY_LEN = 150 


In [ ]:
def compute_rouge_scores(cand_list, ref_list):
    """
    :param cand_list: list of candidate summaries
    :param ref_list: list of reference summaries
    :return: rouge scores
    """
    rouge = Rouge()
    rouge_1_f_score = 0.
    rouge_2_f_score = 0.
    rouge_L_f_score = 0.

    rouge_1_r_score = 0.
    rouge_2_r_score = 0.
    rouge_L_r_score = 0.

    rouge_1_p_score = 0.
    rouge_2_p_score = 0.
    rouge_L_p_score = 0.

    doc_count = len(cand_list)

    for cand, ref in zip(cand_list, ref_list):
        rouge_scores = rouge.get_scores(cand, ref)[0]
        rouge_1_f_score += rouge_scores['rouge-1']['f']
        rouge_2_f_score += rouge_scores['rouge-2']['f']
        rouge_L_f_score += rouge_scores['rouge-l']['f']

        rouge_1_r_score += rouge_scores['rouge-1']['r']
        rouge_2_r_score += rouge_scores['rouge-2']['r']
        rouge_L_r_score += rouge_scores['rouge-l']['r']

        rouge_1_p_score += rouge_scores['rouge-1']['p']
        rouge_2_p_score += rouge_scores['rouge-2']['p']
        rouge_L_p_score += rouge_scores['rouge-l']['p']
    rouge_1_f_score = rouge_1_f_score / doc_count
    rouge_2_f_score = rouge_2_f_score / doc_count
    rouge_L_f_score = rouge_L_f_score / doc_count

    results_dict = {}
    results_dict['rouge_1_f_score'] = rouge_1_f_score
    results_dict['rouge_2_f_score'] = rouge_2_f_score
    results_dict['rouge_l_f_score'] = rouge_L_f_score

    return results_dict

In [10]:
dev_dataset = pd.read_csv('/content/drive/My Drive/W266/data/1024_tokens/ICSI_1024_dev.csv',encoding='latin-1')

dev_dataset = dev_dataset.dropna(subset=['abstractive'])
dev_dataset = dev_dataset.reset_index(drop=True)
dev_dataset.original = 'summarize: ' + dev_dataset.original

print(len(dev_dataset))



2210


In [59]:
MODEL_NAME = "T5_1024_MSFT_AMI"
i = 8
final_df = pd.read_csv('/content/drive/My Drive/W266/results/'+MODEL_NAME + "_epoch" +str(i)+'.csv')

In [60]:
print(len(final_df))

2210


In [61]:
merged_df = pd.concat([dev_dataset.meeting, final_df.Generated_Abstractive_Summary], axis=1)

In [63]:
merged_df["meetinglevel"] = merged_df.meeting.apply(lambda x: x.split(".")[0]) 

In [65]:
abstractive_sentence = set()
gas_list =[]
meeting_list = []
generated_abstractive = ""
for me in set(merged_df.meetinglevel):
  for gas in merged_df[merged_df.meetinglevel == me]['Generated_Abstractive_Summary']:
    generated_abstractive+= gas + " "

  gas_list.append(generated_abstractive)
  meeting_list.append(me)

  abstractive_sentence = set()
  generated_abstractive = " "
  

In [66]:
final_df = pd.DataFrame(
    {'Meeting': meeting_list,
     'Generated_Abstractive_Summary': gas_list
    })

In [72]:
AMI_PATH = "/content/drive/My Drive/W266/data/gold_abstractive_summary/goldsummary_AMI_as_dev.csv"
ICSI_PATH = "/content/drive/My Drive/W266/data/gold_abstractive_summary/goldsummary_ICSI_as_dev.csv"

amigold = pd.read_csv(AMI_PATH)
icsigold = pd.read_csv(ICSI_PATH)

In [76]:
new_df = pd.merge(final_df, amigold,  how='left', left_on='Meeting', right_on ='meeting')


In [78]:
rouge_results = compute_rouge_scores(new_df.Generated_Abstractive_Summary,
                                      new_df.abstractive)

In [79]:
rouge_results

{'rouge_1_f_score': 0.21425437974409214,
 'rouge_2_f_score': 0.06991289662688786,
 'rouge_l_f_score': 0.28485830496864284}

In [80]:
new_df['Generated_Abstractive_Summary'].apply(lambda x: x.split(" ")).map(lambda x: len(x))

0      418
1     1148
2     1185
3      619
4     1099
5      614
6      742
7      551
8     2764
9     1319
10     663
11    1161
12      94
13     527
14    1541
15     273
16     918
17     799
18    1085
19    1257
20    2364
21    1315
22     655
23    1386
24     224
25     114
26    1112
27    1055
28     826
29    1252
30    1349
31     383
32     925
33     682
34     180
35     273
36    1694
37     673
38     249
39     491
40     315
41    1732
42     311
43     999
44     734
45    1229
46    1629
Name: Generated_Abstractive_Summary, dtype: int64

In [81]:
new_df['Generated_Abstractive_Summary'][1]

" The back has a discussion about possibly including a locator function and speech recognition. The industrial designer discussed the interior workings of a remote and then the team discussed the option to include speech recognition in their design. The group decided to make changeable covers in fruit colors and designs and to use one uniform shape. Afterwards, the project manager demonstrated the SmartBoard technology they would be using. The group decided to make changeable covers in fruit colors and designs and to use one uniform shape. The target group goes on to calculate the production cost and finds they are over the budget. compared options for energy sources, materials, case shapes, and interface components such as buttons and LCD screens. deciding that the remote will feature a locator function and will not feature speech recognition. The back has a discussion about possibly including a locator function and speech recognition. The group decided to make changeable covers in fr